# BEE 4750 Lab 2: Uncertainty and Monte Carlo

**Name**:

**ID**:

> **Due Date**
>
> Wednesday, 9/25/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Desktop/Julia Scripts/lab2-ReidHarwell`
   Installed GR_jll ────────────────── v0.73.7+0
   Installed HypergeometricFunctions ─ v0.3.24
   Installed libdecor_jll ──────────── v0.2.2+0
   Installed Opus_jll ──────────────── v1.3.3+0
   Installed libfdk_aac_jll ────────── v2.0.3+0
   Installed Unitful ───────────────── v1.21.0
   Installed StatsFuns ─────────────── v1.3.2
   Installed PtrArrays ─────────────── v1.2.1
   Installed SimpleBufferStream ────── v1.2.0
   Installed PDMats ────────────────── v0.11.31
   Installed Pango_jll ─────────────── v1.54.1+0
   Installed HarfBuzz_jll ──────────── v8.3.1+0
   Installed ColorSchemes ──────────── v3.26.0
   Installed SpecialFunctions ──────── v2.4.0
   Installed GLFW_jll ──────────────── v3.4.0+1
   Installed fzf_jll ───────────────── v0.53.0+0
   Installed Rmath_jll ─────────────── v0.5.1+0
   Installed JLFzf ─────────────────── v0.1.8
   Installed CodecZlib ─────────────── v0.7.6
   Installed Compat ─────────────

In [3]:
using Random # random number generation
using Distributions # probability distributions and interface
using Statistics # basic statistical functions, including mean
using Plots # plotting

## Overview

In this lab, we will use Monte Carlo analysis to analyze a version of
the culmination of every episode of the long-running game show [The
Price Is Right](https://en.wikipedia.org/wiki/The_Price_Is_Right): the
[Showcase](https://en.wikipedia.org/wiki/The_Price_Is_Right#Showcase).

Monte Carlo methods involve the simulation of random numbers from
probability distributions. In an environmental context, we often
propagate these random numbers through some more complicated model and
then compute a resulting statistic which is relevant for assessing
performance or risk, such as an average outcome or a particular
quantile.

You should always start any computing with random numbers by setting a
“seed,” which controls the sequence of numbers which are generated
(since these are not *really* random, just “pseudorandom”). In Julia, we
do this with the `Random.seed!()` function.

In [4]:
Random.seed!(1)

TaskLocalRNG()

It doesn’t matter what seed you set, though different seeds might result
in slightly different values. But setting a seed means every time your
notebook is run, the answer will be the same.

> **Seeds and Reproducing Solutions**
>
> If you don’t re-run your code in the same order or if you re-run the
> same cell repeatedly, you will not get the same solution. If you’re
> working on a specific problem, you might want to re-use
> `Random.seed()` near any block of code you want to re-evaluate
> repeatedly.

### Probability Distributions and Julia

Julia provides a common interface for probability distributions with the
[`Distributions.jl`
package](https://juliastats.org/Distributions.jl/stable/). The basic
workflow for sampling from a distribution is:

1.  Set up the distribution. The specific syntax depends on the
    distribution and what parameters are required, but the general call
    is the similar. For a normal distribution or a uniform distribution,
    the syntax is

    ``` julia
    # you don't have to name this "normal_distribution"
    # μ is the mean and σ is the standard deviation
    normal_distribution = Normal(μ, σ)
    # a is the upper bound and b is the lower bound; these can be set to +Inf or -Inf for an unbounded distribution in one or both directions.
    uniform_distribution = Uniform(a, b)
    ```

    There are lots of both
    [univariate](https://juliastats.org/Distributions.jl/stable/univariate/#Index)
    and
    [multivariate](https://juliastats.org/Distributions.jl/stable/multivariate/)
    distributions, as well as the ability to create your own, but we
    won’t do anything too exotic here.

2.  Draw samples. This uses the `rand()` command (which, when used
    without a distribution, just samples uniformly from the interval
    $[0, 1]$.) For example, to sample from our normal distribution
    above:

    ``` julia
    # draw n samples
    rand(normal_distribution, n)
    ```

Putting this together, let’s say that we wanted to simulate 100
six-sided dice rolls. We could use a [Discrete Uniform
distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.DiscreteUniform).

In [5]:
dice_dist = DiscreteUniform(1, 6) # can generate any integer between 1 and 6
dice_rolls = rand(dice_dist, 100) # simulate rolls

100-element Vector{Int64}:
 1
 3
 5
 4
 6
 2
 5
 5
 5
 2
 ⋮
 3
 6
 5
 5
 6
 3
 6
 6
 6

And then we can plot a histogram of these rolls:

In [6]:
histogram(dice_rolls, legend=:false, bins=6)
ylabel!("Count")
xlabel!("Dice Value")

GKS: dlopen(/Users/rharwell/.julia/artifacts/4cab0d575711b60d292e4bd3088c55d27752a8ea/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/rharwell/.julia/artifacts/4cab0d575711b60d292e4bd3088c55d27752a8ea/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


### Instructions

**Remember to**:

-   Evaluate all of your code cells, in order (using a `Run All`
    command). This will make sure all output is visible and that the
    code cells were evaluated in the correct order.
-   Tag each of the problems when you submit to Gradescope; a 10%
    penalty will be deducted if this is not done.

## Exercise (3 Points)

The Showcase is the final round of every episode of The Price is Right,
matching the two big winners from the episode. Each contestant is shown
a “showcase” of prizes, which are usually some combination of a trip, a
motor vehicle, some furniture, and maybe some other stuff. They then
each have to make a bid on the retail price of the showcase. The rules
are:

-   an overbid is an automatic loss;
-   the contest who gets closest to the retail price wins their
    showcase;
-   if a contestant gets within \$250 of the retail price and is closer
    than their opponent, they win both showcases.

Your goal is to find a wager which maximizes your expected winnings,
which we may as well call utility, based on your assessment of the
probability of your showcase retail price. Make the following
assumptions about your expected winnings if you don’t overbid:

-   If you win both showcases, the value is the double of the single
    showcase value.
-   If you did not win both showcases but bid under the showcase value,
    the probability of being outbid increases linearly as the distance
    between your bid and the value increases (in other words, if you bid
    the exact value, you win with probability 1, and if you bid \$0, you
    win with probability 0).

We’ll assume that the distribution of all showcases offered by the show
is given as truncated normal distribution, which means a normal
distribution which has an upper and/or lower bound. `Distributions.jl`
makes it easy to specify truncations on any distribution, not just
normal distributions. For example, we’ll use the distribution shown in
<a href="#fig-values" class="quarto-xref">Figure 1</a> for the showcase
values (you can just directly use the `showcase_dist` distribution in
your lab code).

In [7]:
showcase_dist = truncated(Normal(31000, 4500), lower=5000, upper=42000)
showcase_samples = rand(showcase_dist, 10_000)
histogram(showcase_samples)

Find your expected winnings if you bid \$35,000. Plot the outcomes of
your Monte Carlo experiment (iterations vs. running estimate). How did
you decide how many samples to use?

In [122]:
#retail prize price comes from truncated distribution
#each contestant must guess as close to their own showcase value without going over
#if your guess is over the actual value of the prize, value earned is 0
#if you guess below the actual value, you have a chance of bid/actual to win the prize value
#if you guess below the actual value, and are within 250, you have a chance of bid/actual to win 2x prize value
#the probability that your guess is closer than opponent is equal to bid/actual 

bidA = 35000
actual_value = 0

function prize_value(bidA) 
    final_value = 0
    actual_value = rand(showcase_dist)
    value_diff = actual_value - bidA
    if (value_diff < 0)
        final_value = 0
    end
    if (value_diff > 0)
        if (value_diff > 250)
            final_value = actual_value * (bidA/actual_value)
        end
        if (value_diff < 250)
            final_value = 2 * actual_value * (bidA/actual_value)
        end
    end
    return final_value
end

samp_size = 10000
prize_list = zeros(samp_size)
prize_runn_avg = zeros(samp_size)
for i = 1:samp_size
    prize_list[i] = prize_value(bidA)
    prize_runn_avg[i] = mean(prize_list[1:i])
end

plot(prize_runn_avg)
plot!(xlabel="Sample Size", ylabel="Running Average Winnings")

## References

Put any consulted sources here, including classmates you worked with/who
helped you.